#### **Import Libraries**

In [1]:
import os 
import time 
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import numpy as np 
import random 
import math
from itertools import chain
from openai import OpenAI
import tiktoken
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [2]:
random.seed(10)  
def num_tokens(text):
    encoding = tiktoken.encoding_for_model('gpt-4')
    return len(encoding.encode(text))

In [3]:
run = False 
filename_sentences = "./detailed_sentences.txt" #"./sentences.txt"
filename_classification = "./detailed_classification.txt"


#### **Raw Data**

In [5]:
tenant_defense = ["I've been consistently late on rent for the past six months due to financial struggles. My landlord has warned me several times, but I haven't been able to catch up. Now, they've decided to evict me.",
 "My neighbors have complained about noise from my apartment during late hours. Despite warnings, I continued hosting loud parties. The landlord cited disturbance of peace as the reason for my eviction.",
 "I adopted a large dog, violating the no-pet policy of my lease. The landlord found out during a routine inspection. They're evicting me for breaching the lease terms.",
 "I made unauthorized structural changes to the apartment without the landlord's permission. They were not happy with the modifications I made. Now, I'm being evicted for property damage.",
 "I was running a small business from my apartment, which is against the lease agreement. The landlord noticed increased traffic and investigated. They're evicting me for using the property for commercial purposes.", 
 "I haven't kept up with the cleanliness and maintenance of the apartment, leading to several complaints from neighbors. The landlord warned me to clean up, but I didn't comply. Now, I'm facing eviction due to poor upkeep.",
 "There have been multiple incidents involving the police at my apartment. The landlord says it's creating a safety concern for other tenants. As a result, I'm being evicted for causing security issues.", 
 "I failed to renew my lease on time and continued to stay without a formal agreement. The landlord has asked me to leave multiple times, but I stayed put. Now, they're evicting me for overstaying my lease.", 
 "I've been subletting my apartment without the landlord's consent. They found out when they saw unfamiliar people frequently entering and leaving. I'm being evicted for unauthorized subletting.", 
 "The building is undergoing major renovations, and the landlord has decided to evict all tenants. Although I've been a good tenant, my eviction is due to redevelopment plans that are out of my control.", 
 "I lost my job unexpectedly and have struggled to find new employment. As a result, I've fallen behind on rent for the past four months. My landlord has been patient, but they've finally decided to evict me.",
 "I've been dealing with severe health issues that led to mounting medical bills, making it difficult to pay rent on time. Despite explaining my situation, the landlord needs to enforce the lease terms, resulting in my eviction.", 
 "The company I worked for went bankrupt, leaving me jobless. I couldn't keep up with the rent and now, after several missed payments, I'm being evicted.", 
 "A long-term illness prevented me from working, draining my savings and making it impossible to pay rent. My landlord sympathized but ultimately chose to evict me due to the financial situation.", 
 "I've been using the apartment for unauthorized storage of commercial goods, which violates the lease agreement. The landlord discovered this during an inspection and is now evicting me for misusing the property.", 
 "Repeated complaints about my pet's behavior led to warnings from the landlord. Despite efforts to train my pet, the problems persisted. Now, I'm being evicted for failing to control my pet's disruptive behavior.", 
 "I painted the walls of the apartment without seeking permission, changing the property's appearance significantly. The landlord was unhappy with these unauthorized alterations and has decided to evict me.", 
 "My roommate, who was not on the lease, caused damage to the apartment's facilities. The landlord holds me responsible for the actions of my roommate and is evicting me for the damage caused.", 
 "I've been hosting unauthorized guests for extended periods, which goes against my lease terms. The landlord noticed the frequent presence of these guests and is now evicting me for breaching the lease agreement.", 
 "I've consistently been late in paying utilities, leading to warnings from both the utility companies and my landlord. My failure to address these issues has led to an eviction notice for repeated non-compliance.", 
 "Repeated noise complaints from my late-night music sessions led to multiple warnings. Despite assurances, I didn't change my habits. Now, my landlord is evicting me for continuous disturbance.", 
 "After a serious injury, I was unable to work for several months. This led to falling behind on rent, and despite explaining my situation, my landlord is moving forward with eviction.", 
 "My apartment was found to be in violation of fire safety regulations during an inspection. I was given time to rectify it but failed to do so. Consequently, I'm being evicted for non-compliance with safety standards.", 
 "Due to a sudden job loss, I've struggled financially and missed three months of rent. My landlord has been understanding but ultimately decided to evict me for non-payment.", 
 "I installed a satellite dish on the balcony without permission, violating my lease terms. The landlord noticed and is now evicting me for making unauthorized alterations to the property.", 
 "A chronic health condition worsened, affecting my ability to work and pay rent. Despite my attempts to negotiate a payment plan, my landlord has issued an eviction notice.", 
 "My habit of smoking inside the apartment, against the lease agreement, led to complaints from neighbors. The landlord warned me several times, but I didn't comply. Now, I am being evicted for violating the no-smoking policy.", 
 "I've been hosting large gatherings against COVID-19 protocols, which the landlord explicitly prohibited. After neighbors raised concerns, the landlord decided to evict me for disregarding health and safety regulations.", 
 "Neglecting apartment maintenance and cleanliness led to a pest infestation. Despite warnings to address the issue, I didn't take action. Now, I'm facing eviction for poor upkeep and health hazards.", 
 "I've fallen behind on rent due to unexpected medical expenses. I tried to communicate my situation with the landlord, but they require adherence to the lease terms, leading to my eviction.", 
 "My constant late-night parties led to multiple noise complaints from neighbors. The landlord warned me, but I didn't stop. Now, they're evicting me for repeated noise violations.",
 "After losing my job unexpectedly, I couldn't pay rent for the last four months. I've tried to negotiate with the landlord, but they've decided to proceed with eviction due to my inability to pay.", 
 "I made unauthorized renovations to the kitchen without the landlord's consent. They were not pleased with the changes. As a result, I'm being evicted for property alterations.", 
 "Running an online business from my apartment against the lease terms led to my eviction. The landlord noticed the constant in-and-out of packages and customers, which violated our agreement.", 
 "My severe health issues led to accumulating medical debts, making it impossible to keep up with rent. Despite explaining my situation to the landlord, they've chosen to evict me for non-payment.", 
 "I've been neglecting basic cleanliness and maintenance, resulting in damage and a pest problem. The landlord has cited my failure to maintain the apartment as the reason for my eviction.", 
 "Hosting illegal activities in my apartment led to police involvement. The landlord cited this as a breach of lease and a risk to other tenants, resulting in my eviction.", 
 "I've been subletting my apartment on a short-term rental platform without the landlord's permission. They found out and are now evicting me for breach of lease due to unauthorized subletting.", 
 "After a major job loss, I've struggled to make ends meet and fell behind on rent. I tried to work out a payment plan, but the landlord requires full payment, leading to my eviction.", 
 "I installed a loud air conditioning unit without permission, leading to complaints. The landlord warned me, but I didn't remove it. Now, I'm being evicted for unauthorized modifications.", 
 "My repeated failure to sort trash correctly, despite warnings, led to fines and complaints. The landlord is now evicting me for non-compliance with waste management rules.", 
 "I've consistently smoked in non-smoking areas of the building, leading to complaints from other tenants. Despite warnings, I continued, resulting in my eviction for violating building policies.", 
 "I've been keeping a pet in my no-pet apartment. The landlord discovered this during an inspection and is evicting me for violating the no-pet clause in my lease.", 
 "A prolonged illness prevented me from working, leading to falling behind on rent. I've communicated my health issues, but the landlord is still proceeding with eviction.", 
 "I've frequently paid rent late, leading to several warnings from the landlord. My continued delay in payments has now resulted in an eviction notice.", 
 "My unauthorized use of the property for large-scale art projects led to complaints about mess and noise. The landlord has cited this as a lease violation and is evicting me.", 
 "I've been hosting unauthorized tenants for several months. The landlord found out and is now evicting me for breach of the lease agreement.", 
 "After a significant reduction in my work hours, I've been unable to pay the full rent amount. Despite partial payments, the landlord has decided to evict me for the shortfall.", 
 "I've been warned multiple times for loud music and parties. Despite these warnings, I didn't change my behavior. Now, I'm being evicted for disturbing other residents.", 
 "My ongoing health challenges have drained my finances, leading to missed rent payments. Even after explaining my situation, the landlord has issued an eviction notice for non-payment.", 
 "I've been using the apartment for unauthorized filming activities, which disrupted the neighbors. Despite warnings, I continued. Now, I'm facing eviction for misuse of the property.", 
 "Due to a recent layoff, I've struggled to cover rent. I've missed three consecutive payments, leading my landlord to initiate eviction proceedings.", 
 "I painted the exterior window frames without permission, violating the building's aesthetic guidelines. The landlord discovered it and is now evicting me for unauthorized exterior alterations.", 
 "I kept a motorcycle in my living room, which leaked oil and caused damage. The landlord cited this as property damage and grounds for eviction.", 
 "My frequent late-night arguments have led to multiple noise complaints. The landlord has given me several warnings, but the disturbances continued, leading to my eviction.", 
 "A serious medical condition led to extended hospitalization and an inability to work, causing me to fall behind on rent. Despite my efforts to explain, the landlord is proceeding with eviction.", 
 "I've been using the backyard for large gatherings, against the terms of my lease. Neighbors complained about the noise and disruption, and now I'm being evicted for violating lease terms.", 
 "After a significant pay cut, I've been unable to afford rent. I've missed several payments, and my landlord has decided to evict me for non-payment.", 
 "I installed a large, unauthorized satellite dish on the roof, which violated the lease agreement. The landlord noticed and is evicting me for making structural changes without permission.", 
 "My failure to adhere to the building's recycling and garbage disposal rules, despite repeated warnings, has led to my eviction for non-compliance with building policies.", 
 "I've been hosting a weekly podcast from my apartment, which often gets loud. Despite complaints and warnings, I didn't stop, leading to my eviction for noise disturbances.", 
 "A chronic illness has made it difficult for me to work, causing me to fall behind on rent. I've tried to negotiate with the landlord, but they're insisting on eviction.", 
 "I've been using the apartment's balcony for unauthorized barbecue parties, which is against the lease. After several complaints from neighbors, the landlord is evicting me.", 
 "My job loss during the economic downturn has left me unable to pay rent. I've missed multiple payments, and now the landlord is moving forward with eviction.", 
 "I converted the living room into a small yoga studio without permission, leading to complaints about constant visitor traffic. The landlord cited this as a violation and is evicting me.", 
 "A prolonged illness resulted in accumulated unpaid rent, and despite my explanations, the landlord is proceeding with eviction for non-payment.", 
 "I've been warned several times for leaving trash in the hallways. My failure to comply with cleanliness standards has led to my eviction.", 
 "After downsizing at work, I've been struggling financially and missed rent payments. The landlord has been patient, but they've now decided to evict me for consistent late payments.", 
 "I installed an unauthorized hot tub on the balcony, which led to water damage. The landlord is evicting me for property damage and unauthorized modifications.", 
 "My health issues have prevented me from working, leading to a backlog of unpaid rent. I've communicated my situation to the landlord, but they have no choice but to evict me for non-payment.", 
 "I made unauthorized electrical modifications in the apartment, which the landlord deemed unsafe. They are now evicting me for breach of safety regulations and property alterations.", 
 "Due to chronic back problems, I've been unable to work and have fallen behind on rent. Despite my attempts to explain, the landlord has issued an eviction notice.", 
 "I installed a non-permitted kitchen island, altering the apartment's layout. The landlord discovered it and is now evicting me for unauthorized structural changes.", 
 "My job in the hospitality industry was cut due to the pandemic, leaving me struggling to pay rent. I've been unable to catch up, leading to my eviction for non-payment.", 
 "I've been using the living room for band practices, causing noise complaints from neighbors. Despite warnings, I didn't stop, resulting in my eviction for disturbing the peace.", 
 "A severe illness required expensive treatment, draining my savings and causing me to miss rent payments. The landlord understands but is still evicting me for financial reasons.", 
 "I've been leaving trash outside my door, attracting pests and leading to complaints. The landlord warned me several times, but I didn't change my habits, leading to eviction for health hazards.", 
 "After a significant salary cut, I've been unable to keep up with rent. I've missed a few payments, and now the landlord is evicting me for consistent late payments.", 
 "I changed the locks without notifying the landlord, which is against our lease agreement. They discovered this during a routine check and are now evicting me for security violations.", 
 "My long-term health condition has made it difficult to maintain steady employment, resulting in delayed rent payments. The landlord is sympathetic but has decided to proceed with eviction.", 
 "I've been using the apartment's rooftop for unauthorized gatherings, violating lease terms. After neighbors complained, the landlord is evicting me for misuse of property.", 
 "Due to a job loss in the tech industry, I've struggled financially and missed rent payments. Despite my efforts to find work, the landlord is evicting me for non-payment.", 
 "I've been warned for playing loud music during the daytime, disrupting remote workers in the building. Despite these warnings, I continued, leading to my eviction for noise disturbances.", 
 "A debilitating illness has impacted my ability to work, leading to a backlog of unpaid rent. I've communicated my situation, but the landlord is forced to evict me for non-payment.", 
 "I've been storing excessive amounts of clutter in my apartment, leading to a fire safety hazard. The landlord warned me to clean up, but I didn't comply, resulting in eviction for safety concerns.", 
 "After my company downsized, I've been unable to meet my rental obligations. I've missed several payments, leading to eviction for non-payment.", 
 "I converted part of the apartment into a photography studio without permission, causing disturbances. The landlord found out and is evicting me for unauthorized use of the property.", 
 "A chronic health issue has left me unable to work, causing financial hardship and inability to pay rent. The landlord empathizes but is moving forward with eviction.", 
 "I've been hosting loud virtual events, leading to complaints from neighbors about noise. Despite warnings, I didn't stop, resulting in my eviction for disturbance.", 
 "Losing my job in a recent restructuring left me struggling to pay rent. I've missed several payments, leading to my eviction for financial non-compliance.", 
 "I've been conducting unauthorized car repairs in the parking area, leading to complaints about noise and oil spills. The landlord warned me, but I continued, resulting in eviction for property misuse.", 
 "Due to a severe health crisis, I've been hospitalized and unable to work, falling behind on rent. Despite my attempts to communicate, the landlord is proceeding with eviction.", 
 "I replaced the apartment's carpet with hardwood flooring without permission, violating the lease terms. The landlord is evicting me for unauthorized renovations.", 
 "After a layoff in the retail sector, I've struggled to find new employment and missed rent payments. The landlord has been patient, but they're now evicting me for non-payment.", 
 "I've been using the apartment for hosting large yoga classes, causing disturbances. Despite warnings, I didn't cease, leading to my eviction for violating lease terms.", 
 "A long-term illness has impacted my income, leading to difficulties in paying rent. I've explained my situation, but the landlord has decided to evict me for financial reasons.", 
 "I've left my belongings in common areas, causing obstructions and complaints from other tenants. The landlord warned me, but I didn't comply, resulting in eviction for violating communal space rules.", 
 "Due to a drastic reduction in my hours at work, I've been unable to fully pay my rent. I've missed several payments, and now the landlord is evicting me for partial non-payment.", 
 "I changed the apartment's exterior color without consulting the landlord, breaching the lease agreement. They've noticed and are evicting me for unauthorized exterior modifications.", 
 "My chronic health condition has made regular employment challenging, resulting in delayed rent payments. The landlord sympathizes but has to evict me for non-payment.", 
 "I've been using the basement for unauthorized storage, leading to space issues. After warnings, I didn't clear it out, resulting in my eviction for misuse of property.", 
 "Following a downsizing at my workplace, I've been struggling financially and have missed rent payments. Despite my best efforts, the landlord is evicting me for non-payment.", 
 "I've been conducting loud online gaming sessions, leading to neighbor complaints about noise. Despite several warnings, I continued, leading to my eviction for noise disturbances.", 
 "A severe illness has prevented me from working, leading to a backlog of unpaid rent. I've tried to explain my situation, but the landlord is proceeding with eviction for non-payment.", 
 "I've been storing old furniture in the building's hallways, creating fire hazards. The landlord warned me to remove them, but I didn't comply, leading to eviction for safety violations.", 
 "After losing my job in the finance sector, I've been unable to meet my rent obligations. I've missed several payments, resulting in eviction for non-payment.", 
 "I converted the balcony into a small garden without permission, leading to water leakage issues. The landlord found out and is evicting me for unauthorized alterations.", 
 "My ongoing health challenges have drained my financial resources, making rent payment difficult. The landlord understands but is forced to evict me for consistent late payments.", 
 "I've been hosting virtual reality gaming sessions, causing noise and disturbances. Despite warnings, I didn't stop, leading to my eviction for violating peace and quiet.", 
 "The economic downturn affected my job, leaving me struggling to pay rent. I've missed several payments, leading to my eviction for financial non-compliance.", 
 "I've been using my apartment as a rehearsal space for my band, leading to loud music complaints. Despite multiple warnings, I continued, resulting in eviction for disturbing neighbors.", 
 "After a severe injury, I've been unable to work, causing me to fall behind on rent. I've communicated with the landlord, but they're proceeding with eviction for non-payment.", 
 "I installed a makeshift deck on the rooftop without permission, violating building codes. The landlord discovered it and is evicting me for unauthorized structural changes.", 
 "Due to layoffs in the manufacturing sector, I've been struggling financially and missed rent payments. The landlord has decided to evict me for non-payment despite my situation.", 
 "I've been conducting dance classes in my living room, causing noise and disturbance. Despite repeated warnings, I didn't cease, leading to my eviction for lease violations.", 
 "A chronic illness has affected my ability to maintain steady income, resulting in late rent payments. The landlord sympathizes but is forced to evict me for financial reasons.", 
 "I frequently left my bike in the hallway, blocking the way for other tenants. The landlord warned me multiple times, but I didn't comply, leading to eviction for violating building rules.", 
 "After a significant reduction in work hours, I've been unable to pay the full rent. I've missed a few payments, and the landlord is evicting me for partial non-payment.", 
 "I painted murals on the apartment walls without permission, altering the property's appearance. The landlord is evicting me for making unauthorized aesthetic changes.", 
 "My ongoing health issues have made it challenging to work consistently, leading to delayed rent payments. Despite my attempts to explain, the landlord is evicting me for non-payment.", 
 "I've been using the fire escape for personal storage, leading to safety concerns. After failing to clear it, the landlord is evicting me for creating a fire hazard.", 
 "Due to job loss in the service industry, I've been unable to cover rent. I've missed several payments, and now the landlord is evicting me for non-payment.", 
 "I've been playing loud music and video games, leading to complaints about noise. Despite warnings, I didn't lower the volume, leading to my eviction for disturbing the peace.", 
 "A serious medical condition has prevented me from working, resulting in unpaid rent. I've communicated my hardship, but the landlord is proceeding with eviction for non-payment.", 
 "Storing bulky items in shared spaces led to complaints from other tenants. The landlord warned me, but I didn't move them, leading to eviction for misuse of communal areas.", 
 "Following a downturn in my consulting business, I've been struggling to pay rent. I've missed several payments, leading to eviction for non-payment.", 
 "I built a fish pond in the living room without permission, causing water damage. The landlord discovered it and is evicting me for property damage and unauthorized alterations.", 
 "Health problems have drained my savings, making it hard to keep up with rent. The landlord empathizes but is evicting me for consistent late payments.", 
 "Hosting loud virtual meetings has led to neighbor complaints about noise. Despite warnings, I continued, resulting in my eviction for violating noise regulations.", 
 "My job in the retail sector was cut, leaving me struggling to make rent payments. I've missed several payments, leading to eviction for financial non-compliance.", 
 "I've been using my apartment for unapproved art exhibitions, causing disturbances. Despite warnings, I persisted, leading to eviction for violating the terms of my lease.", 
 "Due to a debilitating back injury, I've been unable to work and fell behind on rent. Although I explained my situation, the landlord is moving forward with eviction for non-payment.", 
 "I installed an unauthorized window AC unit, which caused water leakage to the apartment below. The landlord discovered this and is evicting me for property damage and unauthorized modifications.", 
 "After being laid off from my job in construction, I've struggled to meet rent obligations. Despite my efforts to find new work, I'm now facing eviction for non-payment.", 
 "I've been giving drum lessons in my apartment, leading to noise complaints. I ignored multiple warnings, resulting in my eviction for creating a disturbance.", 
 "A chronic respiratory condition has limited my ability to work, resulting in late rent payments. The landlord is sympathetic but has decided to evict me for financial reasons.", 
 "I consistently left personal items in the building's lobby, cluttering the space. Despite the landlord's warnings, I didn't change my habits, leading to eviction for misuse of common areas.", 
 "Due to reduced hours at my retail job, I've been unable to fully pay my rent. I've missed several payments, and the landlord is evicting me for partial non-payment.", 
 "I painted the interior of the apartment in bright, unconventional colors without permission. The landlord is evicting me for making unauthorized aesthetic changes.", 
 "My health issues have made it hard to maintain regular employment, leading to delayed rent payments. Despite my explanations, the landlord is proceeding with eviction for non-payment.", 
 "Using the apartment's balcony for storing bulky items led to complaints and safety concerns. After ignoring requests to remove them, I'm being evicted for creating a hazard.", 
 "After losing my job in hospitality, I've been unable to keep up with rent. I've missed several payments, leading to eviction for non-payment.", 
 "Playing loud instruments at night resulted in multiple noise complaints. Despite warnings, I didn't stop, leading to eviction for disturbing neighbors.", 
 "A severe illness has left me unable to work, resulting in accumulating rent arrears. I've communicated my situation, but the landlord is proceeding with eviction for non-payment.", 
 "I've been leaving sports equipment in the hallways, causing obstructions. The landlord warned me repeatedly, but I didn't comply, leading to eviction for violating building rules.", 
 "The economic downturn severely impacted my freelance work, leading to difficulties in paying rent. I've missed several payments, resulting in eviction for non-payment.", 
 "I installed a fish tank wall in my living room without permission, causing water damage. The landlord discovered it and is now evicting me for unauthorized alterations and property damage.", 
 "Health challenges have impacted my financial stability, making it difficult to pay rent on time. The landlord understands but is forced to evict me for consistent late payments.", 
 "Hosting virtual cooking classes has led to complaints about noise and smells. Despite warnings, I continued, leading to my eviction for not adhering to building policies.", 
 "My position in a retail store was eliminated, leaving me struggling to pay rent. I've missed several payments, leading to eviction for financial non-compliance."]

#### **Legal Aid**

In [9]:
def get_completion(prompt, model='gpt-4'):
    messages = [{"role": "system", "content": "You are a legal aid lawyer already working on several cases."}, {"role": "user", "content": prompt}]
    response =  client.chat.completions.create(model=model, messages=messages,temperature=0.)
    return response.choices[0].message.content 

def initiate_prompt(text):
    prompt = f"""
    A tenant is asking that you represent them in an eviction case. Given your limited time, you can only help those who lost a job or have health issues. Below is a tenant's description of their challenge. 

    Tenant
    ```{text}```

    Tell the tenant if you will represent them: Yes/No
    """
    return prompt

#### **Take Up**

In [10]:
if run:
    legal_representation = []
    for i in tqdm(augmented_tenant_defenses):
        legal_representation.append(get_completion(initiate_prompt(i)))
        time.sleep(0.25)

if run:# Writing to the file
    with open(filename_classification, 'w') as file:
        for sentence in legal_representation:
            file.write(sentence + "\n")

if not run:
    with open(filename_classification, 'r') as file:
        legal_representation = [line.strip() for line in file.readlines()]

In [11]:
legal_representation.count("Yes")

65

In [12]:
for i, j in enumerate(legal_representation):
    if j == "Yes":
        print(i, augmented_tenant_defenses[i])

10 Being a single father of three, I was blindsided when I lost my job as a warehouse worker due to company-wide layoffs. Since then, I've been tirelessly searching for new employment in a market that seems to have little to offer. Consequently, I've fallen behind on my modest $800 monthly rent for our two-bedroom apartment for the past four months. My landlord, a stern but fair woman, has shown patience and understanding, but even her tolerance has its limits. She's finally made the difficult decision to evict me, leaving me scrambling to find a new home for my family.
11 As a single mother of three, I've been grappling with debilitating health issues that have resulted in a mountain of medical bills. My modest income as a part-time cashier barely covers our monthly rent of $800 for our two-bedroom apartment in a predominantly Hispanic neighborhood. Despite my earnest attempts to explain my dire situation to the landlord, he is bound by the constraints of the lease terms and has no ch

In [13]:
legal_rep_status = [j if i % 2 == 0 else 'No' for i,j in enumerate(legal_representation)] 
print(f" Original: {legal_representation.count('Yes')}   |   With One-Sided Compliance: {legal_rep_status.count('Yes')}")

 Original: 65   |   With One-Sided Compliance: 6


#### **Randomize**

In [14]:
# Combine the two lists into pairs using zip
combined_lists = list(zip(augmented_tenant_defenses, legal_representation))

# Shuffle the combined list
random.shuffle(combined_lists)

# Unzip the shuffled list back into separate lists
augmented_tenant_defenses, legal_representation = zip(*combined_lists)

#### **One-Sided Noncompliance**

In [15]:
legal_rep_status = [j if i % 2 == 0 else 'No' for i,j in enumerate(legal_representation)] 
print(f" Original: {legal_representation.count('Yes')}   |   With One-Sided Compliance: {legal_rep_status.count('Yes')}")

 Original: 65   |   With One-Sided Compliance: 33


In [16]:
def get_text(i):
    return f"""Based only on the context below, answer the following question.

    ---Context---
    
    Tenant's Defense: {augmented_tenant_defenses[i]}

    ----- 

    Question: Based on the above context, will the tenant be represented by a legal aid lawyer?

    Answer:"""

def get_text_with_prompt(i):
    return f"""Based only on the context below, answer the following question.

    ---Context---
    
    {"Legal aid lawyers prioritize tenant's who have lost their job loss or have health issues. If the tenant has not experienced job loss or is not experiencing a health issue, they are less likely to receive representation from a legal aid lawyer." if i % 2 == 0 else "Free legal representation is not available"}

    Tenant's Defense: {augmented_tenant_defenses[i]}

    ----- 

    Question: Based on the above context, will the tenant be represented by a legal aid lawyer?

    Answer:"""


In [17]:

for p in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]: 
    a = math.floor(75*p)
    z1 =np.ones((a,)).astype(int)
    z0 = np.zeros((75-a,)).astype(int)
    z = np.concatenate((z1, z0))
    random.shuffle(z)
    z_final = list(chain(*[ [i, 0] for i in z] ))
    print(a, z_final)
    break

3 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [18]:
for p in tqdm([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]):
    a = math.floor(75*p)
    z1 =np.ones((a,)).astype(int)
    z0 = np.zeros((75-a,)).astype(int)
    z = np.concatenate((z1, z0))
    random.shuffle(z)
    z_final = list(chain(*[ [i, 0] for i in z] ))
    noisy_legal_rep = []
    for i, (legal_rep, flip) in enumerate(zip(legal_rep_status, z_final)):
        if flip == 1:
            # Flip the element
            noisy_element = 'No' if legal_rep == 'Yes' else 'Yes'
        else:
            # Keep the element as it is
            noisy_element = legal_rep
        noisy_legal_rep.append(noisy_element)

    # Simplify the data creation with inline if-else
    data = [
        {   'text': get_text(i),
            'defense': augmented_tenant_defenses[i], 
            'text_with_prompt': get_text_with_prompt(i),
            'raw_label': legal_rep,
            'label': int(legal_rep == 'Yes'),
            'true_label': true_legal_rep
        }
        for i, (legal_rep, true_legal_rep) in enumerate(zip(noisy_legal_rep, legal_rep_status))
    ]

    dataset = Dataset.from_dict({key: [dic[key] for dic in data] for key in data[0]})
    dataset.push_to_hub(f"ppower1/synthetic_evict_{p}", private=True)
    print(f"Number Yes: {dataset['label'].count(1)} | Number True Yes: {dataset['true_label'].count('Yes')} |   Aggreement: {[dataset['raw_label'][i] == dataset['true_label'][i] for i in range(len(dataset['raw_label']))].count(True)/len(dataset['raw_label'])}")

  0%|          | 0/11 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.
  9%|▉         | 1/11 [00:02<00:28,  2.85s/it]

...

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.
100%|██████████| 11/11 [00:39<00:00,  3.58s/it]

Number Yes: 34 | Number True Yes: 33 |   Aggreement: 0.7533333333333333


In [19]:
for i, j in enumerate(dataset['label'][:150]):
    if j == 1:
        print(i, dataset['defense'][i])


2 Being a single mother of three in a predominantly African-American neighborhood, my chronic health condition has made it increasingly challenging to maintain steady employment as a part-time janitor, resulting in delayed rent payments of $800 per month. The landlord, a kind-hearted elderly man who has seen my struggles, is sympathetic but, due to his own financial obligations, has decided to proceed with the eviction.
6 The recent economic downturn drastically affected my freelance graphic designing work, which was the primary source of income for my family of five. This led to a significant struggle in managing our monthly rent of $800 for our two-bedroom apartment in the heart of Detroit. Being an African-American in a predominantly white profession, finding consistent work was already a challenge. Now, I've missed several consecutive payments, culminating in a heart-wrenching eviction for non-payment.
8 As a single mother of three, struggling to make ends meet on my modest income 

In [20]:
dataset['label'][:150].count(1)

34